In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from pathlib import Path
trainfile = pd.read_csv("../input/google-smartphone-decimeter-challenge/baseline_locations_train.csv")
testfile = pd.read_csv("../input/google-smartphone-decimeter-challenge/baseline_locations_test.csv")
datapath = Path("../input/google-smartphone-decimeter-challenge")
truths = (datapath/'train').rglob('ground_truth.csv')
cols1 = ['collectionName', 'phoneName', 'millisSinceGpsEpoch', 'latDeg',
       'lngDeg']
truthlist = []
for fpath in truths:
    fcsv = pd.read_csv(fpath, usecols = cols1)
    truthlist.append(fcsv)

truthdata = pd.concat(truthlist, ignore_index = True)

In [ ]:
cols2 = ['collectionName','phoneName','millisSinceGpsEpoch','constellationType','svid','signalType','receivedSvTimeInGpsNanos','xSatPosM','ySatPosM','zSatPosM']
logs = (datapath/'train').rglob('*derived.csv')

logslist = []
for fpath in logs:
    logcsv = pd.read_csv(fpath, usecols = cols2)
    logslist.append(logcsv)

logsdata = pd.concat(logslist, ignore_index = True)

In [ ]:
logsdata_svtime = logsdata.groupby(['collectionName','phoneName','millisSinceGpsEpoch']).receivedSvTimeInGpsNanos.apply(lambda x: np.min(x)).reset_index()
logsdata_xsat = logsdata.groupby(['collectionName','phoneName','millisSinceGpsEpoch']).xSatPosM.apply(lambda x: np.average(x)).reset_index()
logsdata_ysat = logsdata.groupby(['collectionName','phoneName','millisSinceGpsEpoch']).ySatPosM.apply(lambda x: np.average(x)).reset_index()
logsdata_zsat = logsdata.groupby(['collectionName','phoneName','millisSinceGpsEpoch']).zSatPosM.apply(lambda x: np.average(x)).reset_index()
logsdata_svid = logsdata.groupby(['collectionName','phoneName','millisSinceGpsEpoch']).svid.apply(lambda x: x.value_counts().index[0]).reset_index()
#logsdata_constellation = logsdata.groupby(['collectionName','phoneName','millisSinceGpsEpoch']).constellationType.apply(lambda x:x.value_counts().index[0]).reset_index()
logsdata_signaltype = logsdata.groupby(['collectionName','phoneName','millisSinceGpsEpoch']).signalType.apply(lambda x: x.value_counts().index[0]).reset_index()

logsdata1 = logsdata_svtime.merge(logsdata_xsat, on = ['collectionName','phoneName','millisSinceGpsEpoch'])
logsdata2 = logsdata1.merge(logsdata_ysat, on = ['collectionName','phoneName','millisSinceGpsEpoch'])
logsdata3 = logsdata2.merge(logsdata_zsat, on = ['collectionName','phoneName','millisSinceGpsEpoch'])
logsdata4 = logsdata3.merge(logsdata_svid, on = ['collectionName','phoneName','millisSinceGpsEpoch'])
#logsdata5 = logsdata4.merge(logsdata_constellation, on = ['collectionName','phoneName','millisSinceGpsEpoch'])
logsdata_final = logsdata4.merge(logsdata_signaltype, on = ['collectionName','phoneName','millisSinceGpsEpoch'])
#logsdata_final['constellationType'] = logsdata_final.constellationType.apply(lambda x: "one" if x == 1 else("two" if x == 2 else("three" if x == 3 else("four" if x == 4 else("five" if x == 5 else("six" if x == 6 else "zero"))))))                                                                                                 
logsdata_final.head()

In [ ]:
print(len(logsdata_final))
print(len(trainfile))
traindata1 = trainfile.merge(truthdata, on = cols1[:3], suffixes=("_current","_truth")) # merge train file with observed data
traindata = pd.merge(traindata1, logsdata_final, how = 'left', on = ['collectionName','phoneName','millisSinceGpsEpoch'])
traindata.sort_values(by = ['phone','millisSinceGpsEpoch'])
#traindata = trainfile.merge(truthdata, on = cols1[:3], suffixes=("_current","_truth")) # merge train file with observed data
#traindata = traindata.merge(logsdata, on = cols1[:3]) # add columns from gnss logs to the traindata
traindata.head()
traindata = traindata.fillna(0)
traindata['latDeg_prev'] = traindata['latDeg_current'].shift(1).where(traindata['phone'].eq(traindata['phone'].shift(1)))
traindata['lngDeg_prev'] = traindata['lngDeg_current'].shift(1).where(traindata['phone'].eq(traindata['phone'].shift(1)))    
traindata['latDeg_next'] = traindata['latDeg_current'].shift(-1).where(traindata['phone'].eq(traindata['phone'].shift(-1)))
traindata['lngDeg_next'] = traindata['lngDeg_current'].shift(-1).where(traindata['phone'].eq(traindata['phone'].shift(-1)))
traindata['time_prev'] = traindata['millisSinceGpsEpoch'].shift(1).where(traindata['phone'].eq(traindata['phone'].shift(1)))
traindata['time_next'] = traindata['millisSinceGpsEpoch'].shift(-1).where(traindata['phone'].eq(traindata['phone'].shift(-1)))
traindata['x_prev'] = traindata['xSatPosM'].shift(1).where(traindata['phone'].eq(traindata['phone'].shift(1)))
traindata['x_next'] = traindata['xSatPosM'].shift(-1).where(traindata['phone'].eq(traindata['phone'].shift(-1)))
traindata['y_prev'] = traindata['ySatPosM'].shift(1).where(traindata['phone'].eq(traindata['phone'].shift(1)))
traindata['y_next'] = traindata['ySatPosM'].shift(-1).where(traindata['phone'].eq(traindata['phone'].shift(-1)))
traindata['z_prev'] = traindata['zSatPosM'].shift(1).where(traindata['phone'].eq(traindata['phone'].shift(1)))
traindata['z_next'] = traindata['zSatPosM'].shift(-1).where(traindata['phone'].eq(traindata['phone'].shift(-1)))

#traindata['svid'] = pd.Categorical(traindata.svid)
#traindata['constellationType'] = pd.Categorical(traindata.constellationType) 
for i in traindata.index:
    if pd.isna(traindata['latDeg_prev'][i]):
        traindata['latDeg_prev'][i] = traindata['latDeg_current'][i]
        traindata['lngDeg_prev'][i] = traindata['lngDeg_current'][i]
        traindata['time_prev'][i] = traindata['millisSinceGpsEpoch'][i]
        traindata['x_prev'][i] = traindata['xSatPosM'][i]
        traindata['y_prev'][i] = traindata['ySatPosM'][i]
        traindata['z_prev'][i] = traindata['zSatPosM'][i]
    if pd.isna(traindata['latDeg_next'][i]):
        traindata['latDeg_next'][i] = traindata['latDeg_current'][i]
        traindata['lngDeg_next'][i] = traindata['lngDeg_current'][i]
        traindata['time_next'][i] = traindata['millisSinceGpsEpoch'][i]
        traindata['x_prev'][i] = traindata['xSatPosM'][i]
        traindata['y_prev'][i] = traindata['ySatPosM'][i]
        traindata['z_prev'][i] = traindata['zSatPosM'][i]
    
traindata['lat_del_prev'] = (traindata['latDeg_current'] - traindata['latDeg_prev'])*10**6
traindata['lat_del_truth'] = (traindata['latDeg_truth'] - traindata['latDeg_prev'])*10**6
traindata['lng_del_prev'] = (traindata['lngDeg_current'] - traindata['lngDeg_prev'])* 10**6
traindata['lng_del_truth'] = (traindata['lngDeg_truth'] - traindata['lngDeg_prev']) * 10**6
traindata['lat_vel'] = ((abs(traindata['latDeg_next'] - traindata['latDeg_prev'])/(traindata['time_next'] - traindata['time_prev'])) * 10**6)
traindata['lng_vel'] = ((abs(traindata['lngDeg_next'] - traindata['lngDeg_prev'])/(traindata['time_next'] - traindata['time_prev'])) * 10**6)
traindata['time_since_last_read'] = traindata['millisSinceGpsEpoch'] - traindata['time_prev']
    #if traindata['time_prev'][i] == traindata['millisSinceGpsEpoch'][i]:
        #traindata['vel_lat'] = 0
        #traindata['vel_lng'] = 0
    #else:
        #traindata['vel_lat'] = (traindata['latDeg_current'][i] - traindata['latDeg_prev'][i])/(traindata['millisSinceGpsEpoch'][i] - traindata['time_prev'][i])
        #traindata['vel_lng'] = (traindata['lngDeg_current'][i] - traindata['lngDeg_prev'][i])/(traindata['millisSinceGpsEpoch'][i] - traindata['time_prev'][i])

print(len(traindata))
#print(len(trainfile))
traindata.head(1)

In [ ]:
#traindata = traindata.fillna(0)
traindata.to_csv('./traindata1.csv', sep = ',', index = False)

In [ ]:
#traindata.columns
from sklearn.model_selection import train_test_split
traindata = traindata.fillna(0)
xcols = [c for c in traindata.columns if c not in ('latDeg_truth','lngDeg_truth','lat_del_truth','lng_del_truth','collectionName','phone','phoneName','millisSinceGpsEpoch','svid',
                                                  'signalType','time_prev','time_next','xSatPosM','ySatPosM','zSatPosM','x_prev','x_next','y_prev','y_next','z_prev','z_next')]
ycols = ['latDeg_truth','lngDeg_truth','lat_del_truth','lng_del_truth']
xdata = traindata[xcols]
ydata = traindata[ycols]
#ydata_lat = traindata[['latDeg_truth']]

xtrain, xtest, ytrain, ytest = train_test_split(xdata, ydata, test_size = 0.3, random_state = 144)
ytrain_lat = ytrain[['lat_del_truth']]
ytrain_lng = ytrain[['lng_del_truth']]
ytest_lat = ytest[['lat_del_truth']]
ytest_lng = ytest[['lng_del_truth']]

#xtrain.to_csv('./xtrain.csv', sep = ',', index = False)
#xtest.to_csv('./xtest.csv', sep = ',', index = False)
#ytrain.to_csv('./ytrain.csv', sep = ',', index = False)
#ytest.to_csv('./ytest.csv', sep = ',', index = False)


In [ ]:
len(xdata.columns)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

def baseline_model():
    model = Sequential()
    model.add(Dense(13, input_dim=13, kernel_initializer='normal', activation='relu'))
    model.add(Dense(6, kernel_initializer = 'normal', activation = 'relu'))
    model.add(Dense(6, kernel_initializer = 'normal', activation = 'relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics=['accuracy'])
    return model

kfold = KFold(n_splits=5)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=1800, batch_size=25)))
#estimators.append(('cv', kfold))
#estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=10)

pipeline = Pipeline(estimators)
ytest['phone'] = traindata['phone']
ytest['millisSinceGpsEpoch'] = traindata['millisSinceGpsEpoch']
ytest['latDeg_prev'] = traindata['latDeg_prev']
ytest['lngDeg_prev'] = traindata['lngDeg_prev']
est_lat = pipeline.fit(xtrain, ytrain_lat)
#results = cross_val_score(pipeline, xtrain, ytrain_lat, cv=kfold)
#print("Baseline: %.5f (%.5f) MSE" % (results.mean(), results.std()))
ypred_lat = est_lat.predict(xtest)
ypred_lat_final = est_lat.predict(testdata_x)
lat_preds_final = testdata[['phone','millisSinceGpsEpoch','latDeg_prev']]
lat_preds_final['lat_del_pred'] = pd.Series(ypred_lat_final)
ytest.reset_index(inplace = True)
ytest.drop(['index'], axis = 1)
ytest['lat_del_pred'] = pd.Series(ypred_lat)
ytest['latDeg_pred'] = ytest['latDeg_prev'] + (ytest['lat_del_pred'] / 10**6)
lat_preds_final['latDeg'] = lat_preds_final['latDeg_prev'] + (lat_preds_final['lat_del_pred']/10**6) 
ytest.to_csv('./lat_preds_data_dum_test.csv', sep = ',', index = False)
lat_preds_final.to_csv('./lat_preds_final.csv')

In [ ]:
testdata_x.head()

In [ ]:
ytest.head()
ytest.to_csv('./ytest_1.csv', sep = ',')

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

def baseline_model():
    model = Sequential()
    model.add(Dense(13, input_dim=13, kernel_initializer='normal', activation='relu'))
    #model.add(Dense(12, kernel_initializer = 'normal', activation = 'relu'))
    model.add(Dense(6, kernel_initializer = 'normal', activation = 'relu'))
    model.add(Dense(6, kernel_initializer = 'normal', activation = 'relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics=['accuracy'])
    return model

kfold = KFold(n_splits=5)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=2000, batch_size=32)))
#estimators.append(('cv', kfold))
#estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=10)

pipeline = Pipeline(estimators)
est_lng = pipeline.fit(xtrain, ytrain_lng)
ypred_lng = est_lng.predict(xtest)
ypred_lng_final = est_lng.predict(testdata_x)
ytest_final = testdata[['phone','millisSinceGpsEpoch','lngDeg_prev']]
ytest['phone'] = traindata['phone']
ytest['millisSinceGpsEpoch'] = traindata['millisSinceGpsEpoch']
ytest['latDeg_prev'] = traindata['latDeg_prev']
ytest['lngDeg_prev'] = traindata['lngDeg_prev']
ytest.reset_index(inplace = True)
ytest.drop(['index'], axis = 1)
ytest['lng_del_pred'] = pd.Series(ypred_lng)
ytest['lngDeg_pred'] = ytest['lngDeg_prev'] + (ytest['lng_del_pred'] / 10**6)
ytest_final['lng_del_pred'] = pd.Series(ypred_lng_final)
ytest_final['lngDeg_pred'] = ytest_final['lngDeg_prev'] + (ytest_final['lng_del_pred']/ 10**6)
ytest.to_csv('./lng_preds_data_dum_test_v1.csv', sep = ',', index = False)
ytest_final.to_csv('./lng_preds_final_v1.csv', sep = ',', index = False)

In [ ]:
#print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))
#ytest.head()
#ytest['lngDeg_pred'] = ytest['lngDeg_truth']
lat_pred = pd.read_csv('/kaggle/input/predictions_v1/lat_preds_final_v1.csv', sep = ',')
lat_pred.head()

In [ ]:
from math import radians, cos, sin, asin, sqrt
def haversine(df):
    dist = []
    for i in df.index:
        lat1 = df['latDeg_truth'][i]
        lng1 = df['lngDeg_truth'][i]
        lat2 = df['latDeg_pred'][i]
        lng2 = df['lngDeg_pred'][i]
        # convert decimal degrees to radians 
        lat1,lng1,lat2,lng2 = map(radians, [lat1, lng1, lat2, lng2])

        # haversine formula 
        dlon = lng2 - lng1 
        dlat = lat2 - lat1 
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = 2 * asin(sqrt(a)) 
        r = 6371000 # Radius of earth in kilometers. Use 3956 for miles
        d = c * r
        dist.append(d)
    return dist

In [ ]:
ytest['dist'] = haversine(ytest)
dist_by_phone = ytest.groupby(['phone']).dist.apply(lambda x: (np.percentile(x,50) + np.percentile(x,95))/2).reset_index()
dist_by_phone.head()

In [ ]:
dist_by_phone['dist'].mean()

In [ ]:
cols2 = ['collectionName','phoneName','millisSinceGpsEpoch','constellationType','svid','signalType','receivedSvTimeInGpsNanos','xSatPosM','ySatPosM','zSatPosM']
logs = (datapath/'test').rglob('*derived.csv')

logslist = []
for fpath in logs:
    logcsv = pd.read_csv(fpath, usecols = cols2)
    logslist.append(logcsv)

logsdata_test = pd.concat(logslist, ignore_index = True)

In [ ]:
logsdata_svtime = logsdata_test.groupby(['collectionName','phoneName','millisSinceGpsEpoch']).receivedSvTimeInGpsNanos.apply(lambda x: np.min(x)).reset_index()
logsdata_xsat = logsdata_test.groupby(['collectionName','phoneName','millisSinceGpsEpoch']).xSatPosM.apply(lambda x: np.average(x)).reset_index()
logsdata_ysat = logsdata_test.groupby(['collectionName','phoneName','millisSinceGpsEpoch']).ySatPosM.apply(lambda x: np.average(x)).reset_index()
logsdata_zsat = logsdata_test.groupby(['collectionName','phoneName','millisSinceGpsEpoch']).zSatPosM.apply(lambda x: np.average(x)).reset_index()
logsdata_svid = logsdata_test.groupby(['collectionName','phoneName','millisSinceGpsEpoch']).svid.apply(lambda x: x.value_counts().index[0]).reset_index()
#logsdata_constellation = logsdata_test.groupby(['collectionName','phoneName','millisSinceGpsEpoch']).constellationType.apply(lambda x:x.value_counts().index[0]).reset_index()
logsdata_signaltype = logsdata_test.groupby(['collectionName','phoneName','millisSinceGpsEpoch']).signalType.apply(lambda x: x.value_counts().index[0]).reset_index()

logsdata1 = logsdata_svtime.merge(logsdata_xsat, on = ['collectionName','phoneName','millisSinceGpsEpoch'])
logsdata2 = logsdata1.merge(logsdata_ysat, on = ['collectionName','phoneName','millisSinceGpsEpoch'])
logsdata3 = logsdata2.merge(logsdata_zsat, on = ['collectionName','phoneName','millisSinceGpsEpoch'])
logsdata4 = logsdata3.merge(logsdata_svid, on = ['collectionName','phoneName','millisSinceGpsEpoch'])
#logsdata5 = logsdata4.merge(logsdata_constellation, on = ['collectionName','phoneName','millisSinceGpsEpoch'])
logsdata_test_final = logsdata4.merge(logsdata_signaltype, on = ['collectionName','phoneName','millisSinceGpsEpoch'])
#logsdata_final['constellationType'] = logsdata_final.constellationType.apply(lambda x: "one" if x == 1 else("two" if x == 2 else("three" if x == 3 else("four" if x == 4 else("five" if x == 5 else("six" if x == 6 else "zero"))))))                                                                                                 
logsdata_test_final.head()


In [ ]:
testdata = pd.DataFrame()
print(len(testfile))
print(len(logsdata_test_final))
testdata = pd.merge(testfile, logsdata_test_final, how = 'left', on = ['collectionName','phoneName','millisSinceGpsEpoch'])
print(len(testdata))
testdata.tail()

In [ ]:
testdata.rename(columns = {'latDeg': 'latDeg_current', 'lngDeg': 'lngDeg_current'}, inplace = True)
testdata.sort_values(by = ['phone','millisSinceGpsEpoch'])
#testdata = testdata.fillna(0)
testdata = testdata.fillna(0)
testdata['latDeg_prev'] = testdata['latDeg_current'].shift(1).where(testdata['phone'].eq(testdata['phone'].shift(1)))
testdata['lngDeg_prev'] = testdata['lngDeg_current'].shift(1).where(testdata['phone'].eq(testdata['phone'].shift(1)))    
testdata['latDeg_next'] = testdata['latDeg_current'].shift(-1).where(testdata['phone'].eq(testdata['phone'].shift(-1)))
testdata['lngDeg_next'] = testdata['lngDeg_current'].shift(-1).where(testdata['phone'].eq(testdata['phone'].shift(-1)))
testdata['time_prev'] = testdata['millisSinceGpsEpoch'].shift(1).where(testdata['phone'].eq(testdata['phone'].shift(1)))
testdata['time_next'] = testdata['millisSinceGpsEpoch'].shift(-1).where(testdata['phone'].eq(testdata['phone'].shift(-1)))
testdata['x_prev'] = testdata['xSatPosM'].shift(1).where(testdata['phone'].eq(testdata['phone'].shift(1)))
testdata['x_next'] = testdata['xSatPosM'].shift(-1).where(testdata['phone'].eq(testdata['phone'].shift(-1)))
testdata['y_prev'] = testdata['ySatPosM'].shift(1).where(testdata['phone'].eq(testdata['phone'].shift(1)))
testdata['y_next'] = testdata['ySatPosM'].shift(-1).where(testdata['phone'].eq(testdata['phone'].shift(-1)))
testdata['z_prev'] = testdata['zSatPosM'].shift(1).where(testdata['phone'].eq(testdata['phone'].shift(1)))
testdata['z_next'] = testdata['zSatPosM'].shift(-1).where(testdata['phone'].eq(testdata['phone'].shift(-1)))

#traindata['svid'] = pd.Categorical(traindata.svid)
#traindata['constellationType'] = pd.Categorical(traindata.constellationType) 
for i in testdata.index:
    if pd.isna(testdata['latDeg_prev'][i]):
        testdata['latDeg_prev'][i] = testdata['latDeg_current'][i]
        testdata['lngDeg_prev'][i] = testdata['lngDeg_current'][i]
        testdata['time_prev'][i] = testdata['millisSinceGpsEpoch'][i]
        testdata['x_prev'][i] = testdata['xSatPosM'][i]
        testdata['y_prev'][i] = testdata['ySatPosM'][i]
        testdata['z_prev'][i] = testdata['zSatPosM'][i]
    if pd.isna(testdata['latDeg_next'][i]):
        testdata['latDeg_next'][i] = testdata['latDeg_current'][i]
        testdata['lngDeg_next'][i] = testdata['lngDeg_current'][i]
        testdata['time_next'][i] = testdata['millisSinceGpsEpoch'][i]
        testdata['x_prev'][i] = testdata['xSatPosM'][i]
        testdata['y_prev'][i] = testdata['ySatPosM'][i]
        testdata['z_prev'][i] = testdata['zSatPosM'][i]
    
testdata['lat_del_prev'] = (testdata['latDeg_current'] - testdata['latDeg_prev'])*10**6
testdata['lng_del_prev'] = (testdata['lngDeg_current'] - testdata['lngDeg_prev'])* 10**6
testdata['lat_vel'] = ((abs(testdata['latDeg_next'] - testdata['latDeg_prev'])/(testdata['time_next'] - testdata['time_prev'])) * 10**6)
testdata['lng_vel'] = ((abs(testdata['lngDeg_next'] - testdata['lngDeg_prev'])/(testdata['time_next'] - testdata['time_prev'])) * 10**6)
testdata['time_since_last_read'] = testdata['millisSinceGpsEpoch'] - testdata['time_prev']
    #if traindata['time_prev'][i] == traindata['millisSinceGpsEpoch'][i]:
        #traindata['vel_lat'] = 0
        #traindata['vel_lng'] = 0
    #else:
        #traindata['vel_lat'] = (traindata['latDeg_current'][i] - traindata['latDeg_prev'][i])/(traindata['millisSinceGpsEpoch'][i] - traindata['time_prev'][i])
        #traindata['vel_lng'] = (traindata['lngDeg_current'][i] - traindata['lngDeg_prev'][i])/(traindata['millisSinceGpsEpoch'][i] - traindata['time_prev'][i])

print(len(testdata))
#print(len(trainfile))
testdata.head(1)

In [ ]:
testdata = testdata.fillna(0)
xcols = [c for c in traindata.columns if c not in ('latDeg_truth','lngDeg_truth','lat_del_truth','lng_del_truth','collectionName','phone','phoneName','millisSinceGpsEpoch','svid',
                                                  'signalType','time_prev','time_next','xSatPosM','ySatPosM','zSatPosM','x_prev','x_next','y_prev','y_next','z_prev','z_next')]

testdata_x = testdata[xcols]
testdata_x.head(1)
testdata_x.to_csv('./testdata.csv', sep = ',')

In [ ]:
testdata['zSatPosM'].value_counts()

In [ ]:
import pandas as pd
lat_preds = pd.read_csv('../input/predictions/lat_preds_final.csv', sep = ',')
lng_preds = pd.read_csv('../input/predictions/lng_preds_final_1.csv', sep = ',')
preds_df = pd.merge(lat_preds, lng_preds, on = ['phone','millisSinceGpsEpoch'])
for i in preds_df.index:
    if pd.isna(preds_df['latDeg'][i]):
        preds_df.at[i,'latDeg'] = preds_df['latDeg_prev'][i]
    if pd.isna(preds_df['lngDeg_pred'][i]):
        preds_df.at[i,'lngDeg_pred'] = preds_df['lngDeg_prev'][i]
        
final_df = preds_df[['phone','millisSinceGpsEpoch','latDeg','lngDeg_pred']]
len(final_df)
final_df.to_csv('./predictions_v10.csv', sep = ',')

In [ ]:
preds_df['latDeg'][3487]